# Shadab Alam: To debug the assignment propagation


In [1]:
using Pkg
Pkg.activate("../.")
#include("srg/MockFBA.jl")
import MockFBA

  Activating project at `~/Documents/Projects/MockFBA`
┌ Info: Precompiling MockFBA [8f5b17de-7982-4597-b420-356a140355ec]
└ @ Base loading.jl:1423


In [2]:
using Printf
using JLD2
using Plots
using BenchmarkTools
using YAML
using FITSIO
using Dates

In [3]:
config=YAML.load_file("../config_tmp.yaml")
tile_date="2019-09-16T00:00:00"
plate_scale=nothing;fp_dic=nothing;exc_dic=nothing


In [25]:
tile_pass=1
tile_index=1
tiles_dic=MockFBA.load_DESI_tiles(config["TILES"]["tile_file"],config["TILES"]["PROGRAM"],
                  tile_pass,config["sky"])
tile_id=tiles_dic["TILEID"][tile_index]
columns=["RA","DEC"]
num_fba=100

100

In [92]:


#targets_dic,fp_dic,exc_dic=MockFBA.Prepare_tile(config,tile_index,tiles_dic;tile_date=tile_date,
#        plate_scale=plate_scale,fp_dic=fp_dic,exc_dic=exc_dic,num_fba=config["NumFBARealization"])

targets_dic=MockFBA.load_targets_intile(config,tile_id,tile_pass,config["group"];
                    columns=["X_UNIT","Y_UNIT","Z_UNIT","PRIORITY"],num_fba=num_fba)

plate_scale=MockFBA.load_platescale(config["focal_plane"]["focalplane_dir"])
telra=tiles_dic["RA"][tile_index]
teldec=tiles_dic["DEC"][tile_index]
targets_dic["X_FP"], targets_dic["Y_FP"]= MockFBA.xyz_unit_2_xy_focalplane(telra, teldec,
    targets_dic["X_UNIT"],targets_dic["Y_UNIT"],targets_dic["Z_UNIT"],plate_scale)


#define the grid scale
grid_scale=MockFBA.get_grid_scale(fp_dic)

#get the focal plane bound
targets_dic["fp_bound"]=MockFBA.get_bound_focal_plane(targets_dic["X_FP"], targets_dic["Y_FP"],grid_scale;pad=1.0)

#Assign zones
targets_dic["zone_x"],targets_dic["zone_y"]=MockFBA.Assign_xy_zone(targets_dic["X_FP"], targets_dic["Y_FP"],targets_dic["fp_bound"])

#Assifn zones for hardware in fp
#fp_dic["zone_x"],fp_dic["zone_y"]=Assign_xy_zone(fp_dic["OFFSET_X"], fp_dic["OFFSET_Y"],targets_dic["fp_bound"])

#sort targets
#MockFBA.sort_targets_ZonePriority!(targets_dic)
#sort devices
#MockFBA.sort_hardware_ZonePriority!(fp_dic,targets_dic["fp_bound"])


zone_xy=zeros(eltype(targets_dic["zone_x"]),targets_dic["ntarget"])
MockFBA.get_zonexy!(targets_dic["zone_x"],targets_dic["zone_y"],targets_dic["fp_bound"],zone_xy)

#Actual sorting, remapping and zone_map consistinting of indices range
#MockFBA.sort_ZoneXp_Apply!(zone_xy,targets_dic["PRIORITY"],targets_dic)


print("Finished")

Debug:Load_tracers_intile QSO
Debug:Load_tracers_intile ELG
Debug:Load_tracers_intile LRG
Finished

In [93]:
function sort_ZoneXp_Apply!(zone_xy,Xp,tdic)

    #count number of entries
    nxp=size(Xp,1)


    #get the range of Xp
    xp_min=minimum(Xp)
    xp_max=maximum(Xp)

    #new quantity for sorting
    #The factor of 0.5 is to make sure the maximum value of priority is lower than next zone
    #It can be any value below 1.0, 0.5 is just a conveninet choice
    #This sort of decide the resolution of values in XP
    new_q=(Xp .- xp_min).*(0.5/(xp_max-xp_min))
    new_q=new_q+zone_xy
    izone_sort=sortperm(new_q,rev=:true)

    modified_ptr=[] #keep track of al the arrays sorted already to not apply sorting twice
    #Now apply this to all of the target arrays
    for tkey in keys(tdic)
        if(tdic[tkey] isa Union{BitArray,Array})
            #this_ptr=pointer_from_objref(tdic[tkey])
            this_ptr=repr(UInt64(pointer_from_objref(tdic[tkey])))
            if((size(tdic[tkey],1)==nxp) & (!(this_ptr in modified_ptr)) )
                if(tdic[tkey] isa Union{BitMatrix,Matrix})
                    tdic[tkey] .= tdic[tkey][izone_sort,:]
                else
                    tdic[tkey] .= tdic[tkey][izone_sort]
                end
                append!(modified_ptr,[this_ptr])
                println("sorting $(tkey) $(this_ptr)")
            else
                println("not sorting ",tkey,size(tdic[tkey],1), this_ptr)
            end
        else
            println("not sorting not array ",tkey)
        end
    end

    return izone_sort
    #Now generate the map from of indices for each zone
    #this_ptr=pointer_from_objref(zone_xy)
    this_ptr=repr(UInt64(pointer_from_objref(zone_xy)))
    if(!(this_ptr in modified_ptr) )
        zone_xy .= zone_xy[izone_sort]
        append!(modified_ptr,[this_ptr])
        println("sorting zone_xy $(this_ptr)")
    end

    uszone=sort(unique(zone_xy),rev=false)
    first_zone=indexin(uszone,zone_xy)
    last_zone=(nxp+1) .- indexin(uszone,reverse(zone_xy))

    #generate the zone index map
    nunique_zone=size(uszone,1)

    tdic["zone_map"]=Dict()
    uu=1
    tdic["zone_map"][uszone[uu]]=[first_zone[uu],last_zone[uu]]
    max_per_zone=tdic["zone_map"][uszone[uu]][2]-tdic["zone_map"][uszone[uu]][1]
    min_per_zone=max_per_zone
    for uu in 2:nunique_zone
        tdic["zone_map"][uszone[uu]]=[first_zone[uu],last_zone[uu]]
        this_zone=last_zone[uu]-first_zone[uu]+1
        if(this_zone<min_per_zone)
            min_per_zone=this_zone
        elseif(this_zone>max_per_zone)
            max_per_zone=this_zone
        end
    end

    tdic["zone_map_keys"]=keys(tdic["zone_map"])
    tdic["count_maxper_zone"]=max_per_zone
    tdic["count_minper_zone"]=min_per_zone
end

iz=sort_ZoneXp_Apply!(zone_xy,targets_dic["PRIORITY"],targets_dic)

sorting zone_y 0x000000010e3fdde0
sorting Z_UNIT 0x000000010e3d55a0
not sorting not array fp_bound
sorting X_FP 0x000000010e3fd000
sorting collided 0x000000010e3d4a90
sorting Y_UNIT 0x000000010e3d4af0
sorting Y_FP 0x000000010e3fd150
sorting zone_x 0x000000010e3fdcc0
sorting X_UNIT 0x000000010e3d4ac0
sorting PRIORITY 0x000000010e3d5660
sorting filekey 0x000000010eb75dd0
not sorting not array file_map
not sorting not array ntarget
sorting index 0x000000010e3d4a00


32256-element Vector{Int64}:
  1404
 27469
  4084
  8082
 26313
 26572
 10428
  3047
 27317
 27716
  8088
 10096
 25058
     ⋮
 21160
 21275
  2606
 30440
 16469
 21110
 22349
 22646
 15486
 15947
 16680
 24389

In [78]:
targets_dic["collided"] isa Union{BitArray,Array}

true

In [77]:
targets_dic["collided"] isa Vector

false

In [6]:
sum(targets_dic["collided"][:,1])

4606

In [104]:

QSO_tile=MockFBA.Load_tracers_intile(config,"ELG",tile_id,tile_pass,config["group"],columns;numobs_need=1,num_fba=num_fba)

Debug:Load_tracers_intile ELG


Dict{Any, Any} with 7 entries:
  "ref_group" => "SGC"
  "PASS"      => 1
  "RA"        => Any[338.87, 338.689, 338.623, 339.299, 339.059, 338.404, 338.6…
  "TILEID"    => 11
  "collided"  => Bool[1 1 … 1 1; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]
  "index"     => Any[5569556, 5569563, 5569571, 5569572, 5569575, 5569580, 5569…
  "DEC"       => Any[0.411751, 0.342537, 0.41185, 0.990852, 0.887901, 0.0526428…

In [108]:
ind_qso_func=QSO_tile["index"][QSO_tile["collided"][:,1]]

1115-element Vector{Any}:
 5569556
 5569563
 5569592
 5569599
 5569603
 5570038
 5570040
 5570069
 5570138
 5570397
 5571984
 5572123
 5572189
       ⋮
 7396982
 7404142
 7411582
 7413969
 7422176
 7427259
 7472075
 7485849
 7489092
 7505605
 7520073
 7531806

In [103]:
function fname_tile(tileid)
    return "/Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/FBA_ph000/TILEID_$(tileid).jld2"
end

f1=jldopen(fname_tile(1),"r")
#f11=jldopen(fname_tile(11),"r")
ind_qso_jld=sort(f1["ELG_index"][f1["ELG_Bool_ass"][:,1]])
close(f1)

In [106]:

ind_qso_coll= (targets_dic["filekey"] .==2) .& (targets_dic["collided"][:,1])
ind_qso_targ=sort(targets_dic["index"][ind_qso_coll])

1115-element Vector{Any}:
 5569556
 5569563
 5569592
 5569599
 5569603
 5570038
 5570040
 5570069
 5570138
 5570397
 5571984
 5572123
 5572189
       ⋮
 7396982
 7404142
 7411582
 7413969
 7422176
 7427259
 7472075
 7485849
 7489092
 7505605
 7520073
 7531806

In [109]:
println(size(ind_qso_func),size(ind_qso_jld),size(ind_qso_targ))

(1115,)(1115,)(1115,)


In [111]:
sum(ind_qso_jld.==ind_qso_targ)

1115

388-element Vector{Any}:
  740525
  740540
  740806
  740832
  741941
  741943
  742937
  743258
  743976
  745990
  746412
  747247
  747252
       ⋮
  994633
  995875
  998325
  998495
  998842
 1000382
 1000420
 1000428
 1000838
 1001335
 1001442
 1003201

In [19]:
keys(targets_dic)

KeySet for a Dict{Any, Any} with 18 entries. Keys:
  "zone_y"
  "Z_UNIT"
  "fp_bound"
  "X_FP"
  "collided"
  "Y_UNIT"
  "Y_FP"
  "zone_map_keys"
  "zone_x"
  "count_maxper_zone"
  "X_UNIT"
  "PRIORITY"
  "zone_map"
  "filekey"
  "file_map"
  "ntarget"
  "index"
  "count_minper_zone"

In [100]:
targets_dic["file_map"]

Dict{Any, Any} with 3 entries:
  2 => ["ELG", "/Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/ELG…
  3 => ["LRG", "/Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/LRG…
  1 => ["QSO", "/Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/QSO…

In [112]:
18842-(3161 +681 +20)

14980

In [113]:
14980-(2884 +277 +4)

11815

In [114]:
1756+435+88+20+4

2303

In [115]:
file=jldopen("/Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/QSO_ph000.jld2","r")

JLDFile /Users/shadab/Documents/Projects/MockFBA/mocks_v2/JLD2_data/QSO_ph000.jld2 (read-only)
 ├─🔢 order
 ├─📂 SGC
 │  ├─🔢 order
 │  ├─📂 zone_77
 │  │  ├─🔢 RA
 │  │  ├─🔢 DEC
 │  │  ├─🔢 PRIORITY
 │  │  └─ ⋯ (14 more entries)
 │  └─ ⋯ (25 more entries)
 └─ ⋯ (3 more entries)

In [117]:
keys(file["SGC"]["zone_77"])

17-element Vector{String}:
 "RA"
 "DEC"
 "PRIORITY"
 "FITS_index"
 "TILEID_PASS0"
 "TILEID_PASS1"
 "TILEID_PASS2"
 "TILEID_PASS3"
 "TILEID_PASS4"
 "TILEID_PASS5"
 "TILEID_PASS6"
 "TILEID_PASS7"
 "TILEID_PASS8"
 "TILEID_PASS9"
 "X_UNIT"
 "Y_UNIT"
 "Z_UNIT"